# Minimal Covid Tracking Project Dataset

In [1]:
from datetime import date, datetime, timedelta


import pandas as pd
import numpy as np

from loguru import logger
import pycountry

%matplotlib inline

# Params 

In [2]:
ctp_dataset = 'https://covidtracking.com/api/v1/states/daily.csv'
google_raw_dataset = 'https://www.gstatic.com/covid19/mobility/Global_Mobility_Report.csv'

minimal_ctp_dataset = '../data/processed/minimal_ctp_covid.csv'

# The Covid Tracking Project Dataset

## Prepare the CTP Dataset 

In [3]:
logger.info('Loading CTP dataset @ %s' % ctp_dataset)

ctp = pd.read_csv(ctp_dataset, parse_dates=['date'], low_memory=False)
ctp.shape

2020-07-28 14:25:42.522 | INFO     | __main__:<module>:1 - Loading CTP dataset @ https://covidtracking.com/api/v1/states/daily.csv


(8081, 41)

In [4]:
ctp = ctp.rename(columns={
    'positive': 'total_cases',
    'death': 'total_deaths',
    'state': 'state_code',
})[['date', 'total_cases', 'total_deaths', 'state_code']].set_index('state_code')

ctp.head(), ctp.date.max()

(                 date  total_cases  total_deaths
 state_code                                      
 AK         2020-07-27       3206.0          21.0
 AL         2020-07-27      81115.0        1491.0
 AR         2020-07-27      39447.0         408.0
 AS         2020-07-27          0.0           0.0
 AZ         2020-07-27     163827.0        3304.0,
 Timestamp('2020-07-27 00:00:00'))

## Get State Populations and Codes

In [5]:
logger.info('Loading population data dataset @ %s' % 'https://www2.census.gov/programs-surveys/popest/datasets/2010-2019/state/detail/SCPRC-EST2019-18+POP-RES.csv')

pops = pd.read_csv('https://www2.census.gov/programs-surveys/popest/datasets/2010-2019/state/detail/SCPRC-EST2019-18+POP-RES.csv', low_memory=False)

pops = pops.rename(columns={
    'NAME': 'state',
    'POPESTIMATE2019': 'population'
})[['state', 'population']].set_index('state')

pops.head()

2020-07-28 14:25:44.309 | INFO     | __main__:<module>:1 - Loading population data dataset @ https://www2.census.gov/programs-surveys/popest/datasets/2010-2019/state/detail/SCPRC-EST2019-18+POP-RES.csv


,population
state,
United States,328239523
Alabama,4903185
Alaska,731545
Arizona,7278717
Arkansas,3017804


In [6]:
logger.info('Loading country codes data dataset @ %s' % 'https://worldpopulationreview.com/states/state-abbreviations')

states = pd.read_html('https://worldpopulationreview.com/states/state-abbreviations')[0]

states = states.rename(columns={
    'State': 'state',
    'Code': 'state_code'
})[['state', 'state_code']].set_index('state')

states.head()

2020-07-28 14:25:44.659 | INFO     | __main__:<module>:1 - Loading country codes data dataset @ https://worldpopulationreview.com/states/state-abbreviations


,state_code
state,
Alabama,AL
Alaska,AK
Arizona,AZ
Arkansas,AR
California,CA


## Combine Datasets

In [7]:
ctp = ctp.join(states.join(pops).reset_index().set_index('state_code')).reset_index().dropna(subset=['state', 'state_code'])

ctp['state_code'].nunique()

51

In [8]:
ctp = ctp.sort_values(by='date')

# Add Missing Cols

## New Cases/Deaths

In [9]:
new_cases_deaths = ctp.groupby('state_code').apply(
    lambda g: g.set_index('date')[['total_cases', 'total_deaths']]\
        .diff()\
        .fillna(g.set_index('date')[['total_cases', 'total_deaths']])).rename(columns={
    'total_cases': 'new_cases',
    'total_deaths': 'new_deaths'
})

ctp = ctp.set_index(['state_code', 'date']).join(new_cases_deaths).reset_index()

ctp.head()

,state_code,date,total_cases,total_deaths,state,population,new_cases,new_deaths
0,WA,2020-01-22,2.0,NaN,Washington,7614893.0,2.0,NaN
1,WA,2020-01-23,2.0,NaN,Washington,7614893.0,0.0,NaN
2,WA,2020-01-24,2.0,NaN,Washington,7614893.0,0.0,NaN
3,WA,2020-01-25,2.0,NaN,Washington,7614893.0,0.0,NaN
4,WA,2020-01-26,2.0,NaN,Washington,7614893.0,0.0,NaN


## Save Raw CTP Dataset as Backup

In [10]:
logger.info('Saving raw CTP dataset @ %s' % '../data/raw/raw_ctp_dataset.csv')

ctp.to_csv('../data/raw/raw_ctp_dataset.csv', index=False)
ctp.shape

2020-07-28 14:25:46.468 | INFO     | __main__:<module>:1 - Saving raw CTP dataset @ ../data/raw/raw_ctp_dataset.csv


(7411, 8)

# The Rt Dataset

## Load the rt.live dataset

In [11]:
# rt = pd.read_csv('https://d14wlfuexuxgcm.cloudfront.net/covid/rt.csv', parse_dates=['date']).rename(columns={
#     'region': 'state_code',
#     'mean': 'rt',
# })[['date', 'state_code', 'rt', 'lower_80', 'upper_80']]

# rt.head()

# The Google Mobility Dataset

In [12]:
logger.info('Loading Google mobility dataset @ %s' % google_raw_dataset)

google = pd.read_csv(google_raw_dataset, parse_dates=['date'], low_memory=False)

google.shape

2020-07-28 14:25:46.969 | INFO     | __main__:<module>:1 - Loading Google mobility dataset @ https://www.gstatic.com/covid19/mobility/Global_Mobility_Report.csv


(713528, 13)

## Save raw data as backup

In [13]:
logger.info('Saving raw Google mobility dataset @ %s' % '../data/raw/raw_google_dataset.csv')

google.to_csv('../data/raw/raw_google_dataset.csv', index=False)
google.shape

2020-07-28 14:25:50.941 | INFO     | __main__:<module>:1 - Saving raw Google mobility dataset @ ../data/raw/raw_google_dataset.csv


(713528, 13)

## State Level Mobility

In [14]:
# Extract the mobility levels for each top-level country
def gen_for_us_states(df):
    
    w = 7
    
    # The columns to use for calculating the mobility drop
    use_cols = ['mobility_retail', 'mobility_transit', 'mobility_work']
    
    # Focus on the US States
    states = df[(df['country_region_code']=='US') & (df['sub_region_1'].notnull()) & (df['sub_region_2'].isnull())].copy()
    
    # Relabel the cols for convenience
    states.columns = ['country_id', 'country', 'state', 'sub_region_2', 'iso_3166_2_code', 'census_fips_code', 'date', 'mobility_retail', 'mobility_grocery', 'mobility_parks', 'mobility_transit', 'mobility_work', 'mobility_home']
    
    # Add rolling cols
    for mobility_col in states.filter(like='mobility_').columns:
        states['rolling_'+mobility_col] = states.groupby(
            'state')[mobility_col].apply(
                lambda s: s.rolling(w).mean())
        
    # Calculate the mobility drop
    states['google_mobility_drop'] = states[use_cols].mean(axis=1)

    
    return states[['country_id', 'country', 'state', 'date', 'google_mobility_drop']]

google_states = gen_for_us_states(google).dropna(subset=['state'])

# Convert mobility drop to mobility level
google_states['google_mobility_level'] = 100+google_states['google_mobility_drop']


google_states.shape, google_states.state.nunique(), google_states.state.unique()

((8058, 6),
 51,
 array(['Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California',
        'Colorado', 'Connecticut', 'Delaware', 'District of Columbia',
        'Florida', 'Georgia', 'Hawaii', 'Idaho', 'Illinois', 'Indiana',
        'Iowa', 'Kansas', 'Kentucky', 'Louisiana', 'Maine', 'Maryland',
        'Massachusetts', 'Michigan', 'Minnesota', 'Mississippi',
        'Missouri', 'Montana', 'Nebraska', 'Nevada', 'New Hampshire',
        'New Jersey', 'New Mexico', 'New York', 'North Carolina',
        'North Dakota', 'Ohio', 'Oklahoma', 'Oregon', 'Pennsylvania',
        'Rhode Island', 'South Carolina', 'South Dakota', 'Tennessee',
        'Texas', 'Utah', 'Vermont', 'Virginia', 'Washington',
        'West Virginia', 'Wisconsin', 'Wyoming'], dtype=object))

# Combine into Minimal Form

In [15]:
ctp_cols = ['new_cases', 'new_deaths', 'population']
google_cols = ['google_mobility_level']

df = ctp.set_index(['state', 'date'])[ctp_cols].join(
    google_states.set_index(['state', 'date'])[google_cols]).reset_index()

df.head()

,state,date,new_cases,new_deaths,population,google_mobility_level
0,Washington,2020-01-22,2.0,NaN,7614893.0,NaN
1,Washington,2020-01-23,0.0,NaN,7614893.0,NaN
2,Washington,2020-01-24,0.0,NaN,7614893.0,NaN
3,Washington,2020-01-25,0.0,NaN,7614893.0,NaN
4,Washington,2020-01-26,0.0,NaN,7614893.0,NaN


## Adjust mobility level so that it is safe to aggregate

In [16]:
# Add a population adjusted mobility level
df['google_mobility_level_by_pop'] = df['google_mobility_level']*df['population']

# Aggregations
We can aggregate across continents and regions but need to be careful about how we combine cols by focusing on this columns that can be added across groupings.

## Aggregations by Continent and Region

In [17]:
def agg_by(df, groupby_cols, aggregation):
    
    agg_cols = ['new_cases', 'new_deaths', 'population']

    agg = df.groupby(groupby_cols)[agg_cols].sum().reset_index()

    # Aggregate mobility separately to account for population and missing values
    agg_mobility = df[df['google_mobility_level'].notnull()].groupby(
        'date')[['google_mobility_level_by_pop', 'population']].sum().reset_index()

    agg_mobility['google_mobility_level'] = agg_mobility['google_mobility_level_by_pop']/agg_mobility['population']

    # Add to the us aggregation
    agg = agg.set_index('date').join(
        agg_mobility.set_index('date')['google_mobility_level']).reset_index()

    return agg

In [18]:
# Safe to add the following cols

by_us = agg_by(df, 'date', 'us')
by_us['aggregation'] = 'us'

df['aggregation'] = df['state']

df_with_aggs = pd.concat([df, by_us], sort=False, ignore_index=True)
df_with_aggs.shape

(7599, 8)

## Rolling Means 

In [19]:
df_with_aggs = df_with_aggs.sort_values(by='date')
df_with_aggs.head()

,state,date,new_cases,new_deaths,population,google_mobility_level,google_mobility_level_by_pop,aggregation
0,Washington,2020-01-22,2.0,NaN,7614893.0,NaN,NaN,Washington
7411,NaN,2020-01-22,2.0,0.0,7614893.0,NaN,NaN,us
7412,NaN,2020-01-23,0.0,0.0,7614893.0,NaN,NaN,us
1,Washington,2020-01-23,0.0,NaN,7614893.0,NaN,NaN,Washington
2,Washington,2020-01-24,0.0,NaN,7614893.0,NaN,NaN,Washington


In [20]:
rolling_cols = ['new_cases', 'new_deaths', 'google_mobility_level']

rolling_means = df_with_aggs.groupby('aggregation').apply(
    lambda g: g[rolling_cols].rolling(7).mean()
).add_suffix('_rolling_mean')

df_with_aggs = pd.concat([df_with_aggs, rolling_means], axis=1)

df_with_aggs.shape

(7599, 11)

## Relative Values

In [21]:
rel_cols = ['new_cases_rolling_mean', 'new_deaths_rolling_mean', 'google_mobility_level_rolling_mean']

relative_values = df_with_aggs.groupby('aggregation').apply(
    lambda g: g[rel_cols]/g[rel_cols].max()
).add_suffix('_rel')
    
df_with_aggs = pd.concat([df_with_aggs, relative_values], axis=1)

df_with_aggs.shape

(7599, 14)

# Save Dataset

In [22]:
logger.info('Saving minimal CTP dataset with mobility data @ %s' % minimal_ctp_dataset)

df_with_aggs.to_csv(minimal_ctp_dataset, index=False)
df_with_aggs.shape, minimal_ctp_dataset, df_with_aggs.date.max()

2020-07-28 14:26:01.160 | INFO     | __main__:<module>:1 - Saving minimal CTP dataset with mobility data @ ../data/processed/minimal_ctp_covid.csv


((7599, 14),
 '../data/processed/minimal_ctp_covid.csv',
 Timestamp('2020-07-27 00:00:00'))